In [1]:
!wget http://cybertron.cg.tu-berlin.de/eitz/projects/classifysketch/sketches_png.zip


--2021-05-10 22:25:34--  http://cybertron.cg.tu-berlin.de/eitz/projects/classifysketch/sketches_png.zip
Resolving cybertron.cg.tu-berlin.de (cybertron.cg.tu-berlin.de)... 130.149.80.52
Connecting to cybertron.cg.tu-berlin.de (cybertron.cg.tu-berlin.de)|130.149.80.52|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 550098987 (525M) [application/zip]
Saving to: ‘sketches_png.zip’

sketches_png.zip    100%[===================>] 524.62M  17.7MB/s    in 34s     

2021-05-10 22:26:08 (15.6 MB/s) - ‘sketches_png.zip’ saved [550098987/550098987]



In [15]:
import math
import random
import numpy as np
from numpy import asarray
import cv2
from PIL import Image, ImageDraw
import sys
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import pickle
import sys
import os
import pathlib
import pickle

In [13]:
with ZipFile('/content/data.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('Data')

In [17]:
import shutil
shutil.make_archive('/content/wheel',
                    'zip',
                    '/content/Folder/png/',
                    'wheel')

# to delete folder
!rm -rf /content/Data/eye

'/content/wheel.zip'

In [99]:
def do_it_all(data_path):
    
    


    image = Image.open(data_path)
    #new_image = image.resize((64, 64))
    image.save('padded_cropped_shrinked_img.png')
    final_img_path = 'padded_cropped_shrinked_img.png'
    # -------------------------------------------------------
    # Collecting data for training


    # load the image
    image = Image.open(final_img_path)
    # convert image to numpy array
    img_data = asarray(image)
    return img_data

In [100]:
def hog_alorithm(training_data):
    training_data_resized = []
    for img in training_data:
        training_data_resized.append(cv2.resize(img, (80, 80)))

    ppc = 16
    hog_images = []
    hog_features = []
    for image in training_data_resized:
        fd,hog_image = hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
        hog_images.append(hog_image)
        hog_features.append(fd)

    print(len(hog_features))
    hog_features = np.array(hog_features)
    return hog_features

In [101]:
def preprocess(img_path): #input an image   

    # load the image
    image = Image.open(img_path)
    # convert image to numpy array
    img_data = asarray(image)

    our_test = []
    our_test.append(img_data)


    our_test_resized = [cv2.resize(our_test[0], (80, 80))]

    ppc = 16
    hog_images = []
    hog_features = []
    for image in our_test_resized:
        fd,hog_image = hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
        hog_images.append(hog_image)
        hog_features.append(fd)

    hog_features = np.array(hog_features)
    return hog_features

In [102]:
training_data = []
classes_sizes = []
labels = []
data_path = "/content/Data"

for class_path in os.listdir(data_path):

    class_path = os.path.join(data_path, class_path)

    class_len = len(os.listdir(class_path))
    classes_sizes.append(class_len)

    for shape_path in os.listdir(class_path):
        print(shape_path)
        shape_path = os.path.join(class_path, shape_path)

        if os.stat(shape_path).st_size == 0:
            os.remove(shape_path)
            continue

        training_data.append(do_it_all(shape_path))

hog_features = hog_alorithm(training_data)

8846.png
8844.png
8826.png
8879.png
8848.png
8853.png
8842.png
8808.png
8867.png
8839.png
8836.png
8817.png
8818.png
8875.png
8816.png
8832.png
8863.png
8821.png
8870.png
8811.png
8880.png
8847.png
8868.png
8857.png
8803.png
8801.png
8878.png
8876.png
8874.png
8854.png
8841.png
8856.png
8845.png
8828.png
8865.png
8860.png
8838.png
8815.png
8810.png
8827.png
8852.png
8829.png
8823.png
8805.png
8835.png
8858.png
8804.png
8859.png
8802.png
8840.png
8864.png
8809.png
8855.png
8861.png
8820.png
8812.png
8819.png
8806.png
8824.png
8833.png
8830.png
8843.png
8851.png
8834.png
8825.png
8813.png
8822.png
8850.png
8871.png
8814.png
8849.png
8866.png
8869.png
8837.png
8807.png
8872.png
8862.png
8873.png
8877.png
8831.png
29.png
73.png
41.png
42.png
64.png
3.png
67.png
62.png
63.png
40.png
15.png
37.png
76.png
7.png
22.png
79.png
46.png
25.png
52.png
51.png
1.png
2.png
45.png
68.png
24.png
58.png
16.png
32.png
34.png
77.png
44.png
56.png
71.png
66.png
19.png
53.png
5.png
47.png
33.png
49.png
55.pn

In [103]:
classes_sizes

[80, 80, 80, 80, 80]

In [104]:
labels = np.zeros((classes_sizes[0], 1), dtype="int8")
for i in range(1, len(classes_sizes)):
    label_i = np.zeros((classes_sizes[i], 1), dtype="int8") + i
    labels = np.concatenate((labels, label_i))



In [105]:
clf = svm.SVC(probability=True)
data_frame = np.hstack((hog_features, labels))
np.random.shuffle(data_frame)

percentage = 80
partition = int(len(hog_features)*percentage/100)

x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()

clf.fit(x_train, y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [106]:
y_pred = clf.predict(x_test)

In [107]:
print(clf.score(x_train,y_train))

0.91875


In [108]:
print(clf.score(x_test,y_test))

0.8375


In [109]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.59      0.77      0.67        13
         1.0       0.93      0.81      0.87        16
         2.0       1.00      0.76      0.87        17
         3.0       0.89      0.84      0.86        19
         4.0       0.83      1.00      0.91        15

    accuracy                           0.84        80
   macro avg       0.85      0.84      0.83        80
weighted avg       0.86      0.84      0.84        80



In [110]:
y_pred = clf.predict(x_test)

In [111]:
# save the model to disk
filename = 'finalized_model.pkl'
pickle.dump(clf, open(filename, 'wb'))

In [112]:
#filename = os.path.join(folder_path, 'finalized_model_v4.pkl')
loaded_model = pickle.load(open("/content/finalized_model.pkl", 'rb'))

In [121]:
final_img_path = "/content/Data/airplane/11.png"
predicted_class = loaded_model.predict(preprocess(final_img_path))
prob_of_classes = loaded_model.predict_proba(preprocess(final_img_path))
prob_of_classes

array([[0.07571795, 0.80881957, 0.01034517, 0.05759563, 0.04752168]])

In [122]:
predicted_class


array([1.])

In [ ]:
airplane = 1
bicycle = 3
t-shirt = 4
tree = 2
house = 0